# Preprocessing

In [ ]:
import pandas as pd

## 1. Files

In [ ]:
file_path = "data/mi_eaglei_outages_2021.csv"
df_eaglei_2021 = pd.read_csv(file_path)

In [ ]:
file_path = "data/mi_eaglei_outages_2022.csv"
df_eaglei_2022 = pd.read_csv(file_path)

In [ ]:
df_eaglei_2021['run_start_time'] = pd.to_datetime(df_eaglei_2021['run_start_time'], errors='coerce')
# sort data by county and timestamp
df_eaglei_2021.sort_values(['county', 'run_start_time'], inplace=True)

min_time_2021 = df_eaglei_2021[df_eaglei_2021['run_start_time'].dt.year == 2021]['run_start_time'].min()
max_time_2021 = df_eaglei_2021[df_eaglei_2021['run_start_time'].dt.year == 2021]['run_start_time'].max()

print("2021 - Minimum run_start_time:", min_time_2021)
print("2021 - Maximum run_start_time:", max_time_2021)

In [ ]:
df_eaglei_2022['run_start_time'] = pd.to_datetime(df_eaglei_2022['run_start_time'], errors='coerce')
# sort data by county and timestamp
df_eaglei_2022.sort_values(['county', 'run_start_time'], inplace=True)

min_time_2022 = df_eaglei_2022[df_eaglei_2022['run_start_time'].dt.year == 2022]['run_start_time'].min()
max_time_2022 = df_eaglei_2022[df_eaglei_2022['run_start_time'].dt.year == 2022]['run_start_time'].max()

print("2022 - Minimum run_start_time:", min_time_2022)
print("2022 - Maximum run_start_time:", max_time_2022)

## 2. NaN Processing

In [ ]:
def fill_short_gaps(df, short_gap_limit=16):
    """
    Fill short gaps (16 hours) in 'sum' using linear interpolation
    """
    df_interpolated = df.copy()
    df_interpolated.sort_values(['fips_code','run_start_time'], inplace=True)
    df_interpolated['sum'] = (
        df_interpolated.groupby('fips_code')['sum']
        .apply(lambda x: x.interpolate(limit=short_gap_limit, limit_direction='forward'))
        .reset_index(level=0, drop=True)  # Reset index
    )

    return df_interpolated

In [ ]:
df_eaglei_2021 = fill_short_gaps(df_eaglei_2021)

df_eaglei_2022 = fill_short_gaps(df_eaglei_2022)

## 3. Aggregation

In [ ]:
def select_hourly_snapshot(df_in, timecol='run_start_time', sumcol='sum'):
    """
    1. Sum 'sum' values for all counties within each 15-min timestamp - total_15.
    2. hour_bin = floor('h') for all timestamps
    3. for each 'hour_bin' keep the raw with max 'total_15'
    4. merge with initial dataframe, keep the raws with the maximum 'total_15'
    5. run_start_time = hour_bin, sum = total_15 and save initial run_start_time in orig_run_start_time.
    """


    df = df_in.copy()
    df[timecol] = pd.to_datetime(df[timecol], errors='coerce')
    df.sort_values(by=timecol, inplace=True)

    # total 'sum' on all counties
    df_sum_15 = (
        df.groupby(timecol, as_index=False)[sumcol]
          .sum()  # sum all raws with the same timecol
          .rename(columns={sumcol: 'total_15'})
    )

    # hour_bin = floor('h') rounded by hour
    df_sum_15['hour_bin'] = df_sum_15[timecol].dt.floor('h')

    # for each 'hour_bin' keep the raw with max total_15
    df_sum_15.sort_values(['hour_bin','total_15'], ascending=[True, True], inplace=True)
    df_max_15 = df_sum_15.drop_duplicates(subset='hour_bin', keep='last').copy()

    #merge with initial dataframe keep the raws where starttime = max load (df_max_15)
    df_merged = pd.merge(
        df,
        df_max_15[[timecol,'hour_bin']],  # merge by run_start_time
        on=timecol,
        how='inner'
    )

    # df_merged contains all columns from df_in + hour_bin
    # save initial run_start_time in orig_run_start_time, and replace run_start_time with hour_bin
    df_merged['orig_run_start_time'] = df_merged[timecol]
    df_merged[timecol] = df_merged['hour_bin']

    # drop hour_bin
    df_merged.drop(columns=['hour_bin'], inplace=True, errors='ignore')

    return df_merged

In [ ]:
df_eaglei_2021['run_start_time'] = pd.to_datetime(df_eaglei_2021['run_start_time'], errors='coerce')
df_eaglei_2021.sort_values(['county','run_start_time'], inplace=True)

df_outages_2021 = select_hourly_snapshot(df_eaglei_2021,
                                             timecol='run_start_time',
                                             sumcol='sum')

print("df_outages_2021 shape:", df_outages_2021.shape)
print(df_outages_2021.head(20))

In [ ]:
df_eaglei_2022['run_start_time'] = pd.to_datetime(df_eaglei_2022['run_start_time'], errors='coerce')
df_eaglei_2022.sort_values(['county','run_start_time'], inplace=True)

df_selected_2022 = select_hourly_snapshot(df_eaglei_2022,
                                             timecol='run_start_time',
                                             sumcol='sum')

print("df_outages_2022 shape:", df_outages_2022.shape)
print(df_outages_2022.head(20))

In [ ]:
df_outages_2021.drop(columns='orig_run_start_time', inplace=True, errors='ignore')


In [ ]:
df_outages_2022.drop(columns='orig_run_start_time', inplace=True, errors='ignore')